# ResNet101

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torchsummary import summary

In [2]:
train_dir = '/home/msvermis/Downloads/ML_projects/satellite-project/Data/resnet_dataset/train'
test_dir = '/home/msvermis/Downloads/ML_projects/satellite-project/Data/resnet_dataset/test'

In [3]:
# Image data augmentation and normalization
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, interpolation=transforms.InterpolationMode.BILINEAR),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation((0, 180)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_transform = transforms.Compose([
    transforms.Resize(232, interpolation=transforms.InterpolationMode.BILINEAR),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [4]:
# Load datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=test_transform)

# Create data loaders
batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=8)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=8)

In [ ]:
g = {'defected': 0, 'normal': 1}

In [6]:
train_dataset.classes

['defected', 'normal']

In [11]:
# Load pre-trained ResNet model
resnet_model = models.resnet101(weights='DEFAULT')

In [12]:
# Freeze the layers of the pre-trained model
for param in resnet_model.parameters():
    param.requires_grad = False

# Modify the classifier part of the model for binary classification
resnet_model.fc = nn.Sequential(
    nn.Linear(in_features=resnet_model.fc.in_features, out_features=10),
    nn.Sigmoid()
)

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(resnet_model.fc.parameters(), lr=0.001)


# Move the model to GPU if available
resnet_model = resnet_model.to(device)

# Print model summary
summary(resnet_model, (3, 480, 480))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 240, 240]           9,408
       BatchNorm2d-2         [-1, 64, 240, 240]             128
              ReLU-3         [-1, 64, 240, 240]               0
         MaxPool2d-4         [-1, 64, 120, 120]               0
            Conv2d-5         [-1, 64, 120, 120]           4,096
       BatchNorm2d-6         [-1, 64, 120, 120]             128
              ReLU-7         [-1, 64, 120, 120]               0
            Conv2d-8         [-1, 64, 120, 120]          36,864
       BatchNorm2d-9         [-1, 64, 120, 120]             128
             ReLU-10         [-1, 64, 120, 120]               0
           Conv2d-11        [-1, 256, 120, 120]          16,384
      BatchNorm2d-12        [-1, 256, 120, 120]             512
           Conv2d-13        [-1, 256, 120, 120]          16,384
      BatchNorm2d-14        [-1, 256, 1

In [13]:
def checkpoint(model, optimizer, filename):
    torch.save({'optimizer': optimizer.state_dict(),
    'model': model.state_dict(),}, filename)
    
def resume(model, filename):
    checkpoint = torch.load(filename)
    model.load_state_dict(checkpoint['model'])
    optimizer.load_state_dict(checkpoint['optimizer'])

In [ ]:
from tqdm.auto import tqdm
import PIL
from torch.utils.tensorboard import SummaryWriter
PIL.Image.MAX_IMAGE_PIXELS = 124010496
writer = SummaryWriter(log_dir="/home/msvermis/Downloads/ML_projects/satellite-project/satellite/NN/logs")

# Train the model
num_epochs = 300
for epoch in tqdm(np.arange(53, num_epochs)):
    if epoch%50 ==0 and epoch!=0:
        checkpoint(resnet_model, optimizer, f'weights/resnet_classification_model_{epoch}.pth')
    resnet_model.train()
    train_loss = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = resnet_model(inputs)
        loss = criterion(outputs.squeeze(), labels.float())
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
    train_loss/=len(train_loader)

    # Evaluate the model on the test set
    resnet_model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = resnet_model(inputs)
            predicted = (outputs.squeeze() > 0.5).float()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        accuracy = correct / total
        print(f'Epoch {epoch+1}/{num_epochs}, Test Accuracy: {accuracy:.4f}')
        writer.add_scalars(main_tag="Loss", tag_scalar_dict={"train_loss": train_loss}, global_step=epoch)
        writer.add_scalars(main_tag="Accuracy", tag_scalar_dict={"test_acc": accuracy}, global_step=epoch)
        #writer.add_figure("Confusion matrix", createConfusionMatrix(test_loader, resnet_model), epoch)
        # Close the writer
        writer.close()

In [17]:
# Viewing TensorBoard in Jupyter and Google Colab Notebooks (uncomment to view full TensorBoard instance)
%tensorboard --logdir logs

UsageError: Line magic function `%tensorboard` not found.


In [18]:
# Save the model
torch.save(resnet_model.state_dict(), 'weights/resnet_classification_model.pth')

### Import resnet model

In [19]:
#import resnet model
import torch
import torch.nn as nn
from torchvision import transforms, models
from PIL import Image

# Set the path to the saved model file
model_path = 'weights/resnet_classification_model.pth'  # Change the filename accordingly

# Create a transformation for input images (should match the one used during training)
transform = transforms.Compose([
    transforms.Resize(232, interpolation=transforms.InterpolationMode.BILINEAR),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the model
model = models.resnet101(weights = 'DEFAULT')  # assuming you saved the entire model including architecture
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 1),
    nn.Sigmoid()
)

#checkpoint = torch.load(model_path)
#model.load_state_dict(checkpoint['model'])
model.load_state_dict(torch.load(model_path))
model.eval()  # Set the model to evaluation mode

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

# EfficientNet 

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torchsummary import summary

In [74]:
import os
import cv2
path_folder = '/home/msvermis/Downloads/ML_projects/satellite-project/Data/efficientnet_dataset'
for root, dirs, files, rootfd in os.fwalk(path_folder):
    if len(files)!=0:
        print(f'folder name: {root}')
        for i in files:
            size = 540
            all_path = root + "/"+ i
            img = cv2.imread(all_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = cv2.resize(img, (size*2, size*2))
            C = range(2)
            for x in C:
                for y in C:
                    cropped = img[x*size:(x+1)*size, y*size:(y+1)*size]
                    save_path = root + "/" + str(x) + str(y) + i
                    cv2.imwrite(save_path, cropped)
            os.remove(all_path)

folder name: /home/msvermis/Downloads/ML_projects/satellite-project/Data/efficientnet_dataset/train/defected
folder name: /home/msvermis/Downloads/ML_projects/satellite-project/Data/efficientnet_dataset/train/normal
folder name: /home/msvermis/Downloads/ML_projects/satellite-project/Data/efficientnet_dataset/test/defected
folder name: /home/msvermis/Downloads/ML_projects/satellite-project/Data/efficientnet_dataset/test/normal


In [6]:
train_dir = '/home/msvermis/Downloads/ML_projects/satellite-project/Data/efficientnet_dataset/train'
test_dir = '/home/msvermis/Downloads/ML_projects/satellite-project/Data/efficientnet_dataset/test'

In [10]:
# Image data augmentation and normalization
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(480, interpolation=transforms.InterpolationMode.BILINEAR),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation((0, 180)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

test_transform = transforms.Compose([
    transforms.Resize(480, interpolation=transforms.InterpolationMode.BILINEAR),
    transforms.CenterCrop(480),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

In [20]:
# Load datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=test_transform)

# Create data loaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=8)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=8)

In [12]:
# Load pre-trained ResNet model
effic_model = models.efficientnet_v2_l(weights='DEFAULT')

In [13]:
# Freeze the layers of the pre-trained model
for param in effic_model.parameters():
    param.requires_grad = False

# Modify the classifier part of the model for binary classification
effic_model.classifier = nn.Sequential(
    nn.Dropout(p=0.2, inplace=True), 
    nn.Linear(in_features=effic_model.classifier[1].in_features, 
                    out_features=1, # same number of output units as our number of classes
                    bias=True),
    nn.Sigmoid())

In [18]:
torch.cuda.empty_cache()

import gc
del model
gc.collect()


1831

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(effic_model.classifier.parameters(), lr=0.001)


# Move the model to GPU if available
effic_model = effic_model.to(device)

# Print model summary
#summary(effic_model, (3, 480, 480))

In [15]:
def checkpoint(model, optimizer, filename):
    torch.save({'optimizer': optimizer.state_dict(),
    'model': model.state_dict(),}, filename)
    
def resume(model, filename):
    checkpoint = torch.load(filename)
    model.load_state_dict(checkpoint['model'])
    optimizer.load_state_dict(checkpoint['optimizer'])

In [21]:
from tqdm.auto import tqdm
import PIL
from torch.utils.tensorboard import SummaryWriter
PIL.Image.MAX_IMAGE_PIXELS = 124010496
writer = SummaryWriter(log_dir="/home/msvermis/Downloads/ML_projects/satellite-project/satellite/NN/logs2")

# Train the model
num_epochs = 200
for epoch in tqdm(range(num_epochs)):
    if epoch%50 ==0 and epoch!=0:
        checkpoint(effic_model, optimizer, f'weights2/efficient_classification_model_{epoch}.pth')
    effic_model.train()
    train_loss = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = effic_model(inputs)
        loss = criterion(outputs.squeeze(), labels.float())
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
    train_loss/=len(train_loader)

    # Evaluate the model on the test set
    effic_model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = effic_model(inputs)
            predicted = (outputs.squeeze() > 0.5).float()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        accuracy = correct / total
        print(f'Epoch {epoch+1}/{num_epochs}, Test Accuracy: {accuracy:.4f}')
        writer.add_scalars(main_tag="Loss", tag_scalar_dict={"train_loss": train_loss}, global_step=epoch)
        writer.add_scalars(main_tag="Accuracy", tag_scalar_dict={"test_acc": accuracy}, global_step=epoch)
        #writer.add_figure("Confusion matrix", createConfusionMatrix(test_loader, effic_model), epoch)
        # Close the writer
        writer.close()

  0%|          | 0/200 [00:03<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 450.00 MiB. GPU 0 has a total capacty of 5.78 GiB of which 236.88 MiB is free. Including non-PyTorch memory, this process has 4.59 GiB memory in use. Of the allocated memory 4.35 GiB is allocated by PyTorch, and 141.11 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [1]:
%tensorboard --logdir logs

UsageError: Line magic function `%tensorboard` not found.


In [92]:
torch.save(effic_model.state_dict(), 'weights/efficient_classification_model.pth')

### Import EfficientNet

In [4]:
#import resnet model
import torch
import torch.nn as nn
from torchvision import transforms, models
from PIL import Image

# Set the path to the saved model file
model_path = 'EfficientNet_v2_L/weights/resnet_classification_model.pth'  # Change the filename accordingly

# Create a transformation for input images (should match the one used during training)
transform = transforms.Compose([
    transforms.Resize(480),
    transforms.CenterCrop(480),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

model = models.efficientnet_v2_l(weights='DEFAULT')
model.classifier = nn.Sequential(
    torch.nn.Dropout(p=0.2, inplace=True), 
    torch.nn.Linear(in_features = model.classifier[1].in_features, 
                    out_features=1, # same number of output units as our number of classes
                    bias=True),
    nn.Sigmoid()
)
#checkpoint = torch.load(model_path)
#model.load_state_dict(checkpoint['model'])
model.load_state_dict(torch.load(model_path))
model.eval()  # Set the model to evaluation mode

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): FusedMBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
      )
      (1): FusedMBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  

# Test the model

In [108]:
#make confusion matrix
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def createConfusionMatrix(loader, model):
    y_pred = [] # save predction
    y_true = [] # save ground truth

    # iterate over data
    for inputs, labels in loader:
        #inputs, labels = inputs.to(device), labels.to(device)
        output = model(inputs)  # Feed Network

        
        output = (output.squeeze() > 0.5).float()
        y_pred.extend(output)  # save prediction

        labels = labels.data.cpu().numpy()
        y_true.extend(labels)  # save ground truth

    # constant for classes
    classes = ('defected', 'normal')

    # Build confusion matrix
    cf_matrix = confusion_matrix(y_true, y_pred)
    df_cm = pd.DataFrame(cf_matrix / np.sum(cf_matrix, axis=1)[:, None], index=[i for i in classes],
                         columns=[i for i in classes])
    # Create Heatmap
    plt.figure(figsize=(12, 7))    
    return sn.heatmap(df_cm, annot=True).get_figure()

In [5]:
#test images from the folder 
import os 

result = 0
# Load and preprocess the input image
image_folder = '/home/msvermis/Downloads/ML_projects/satellite-project/Data/efficientnet_dataset/test/defected'
#image_path = "221102_1200_7.jpg"
images = os.listdir(image_folder)
for filename in images:
    image_path = image_folder + "/" + filename
    image = Image.open(image_path).convert('RGB')
    input_image = transform(image)
    input_image = input_image.unsqueeze(0)  # Add a batch dimension

    # Make predictions
    with torch.no_grad():
        output = model(input_image)

    result+=output.item()
    # Interpret the output
    prediction = 'with defects' if output.item() < 0.5 else 'without defects'
    print(f'The image {filename} is predicted to be: {prediction}')
print(result/len(images))

The image 10000033.jpg is predicted to be: with defects
The image 00001497.jpg is predicted to be: without defects
The image 11001072.jpg is predicted to be: without defects
The image 10001488.jpg is predicted to be: with defects
The image 10000115.jpg is predicted to be: without defects
The image 10000668.jpg is predicted to be: without defects
The image 00001577.jpg is predicted to be: with defects
The image 00001488.jpg is predicted to be: without defects
The image 10im6_def.jpg is predicted to be: with defects
The image 01001445.jpg is predicted to be: without defects
The image 01001848.jpg is predicted to be: with defects
The image 10000198.jpg is predicted to be: without defects
The image 11im7_def.jpg is predicted to be: without defects
The image 00000082.jpg is predicted to be: with defects
The image 01000082.jpg is predicted to be: with defects
The image 00001286.jpg is predicted to be: without defects
The image 10001441.jpg is predicted to be: without defects
The image 000002

In [106]:
torch.cuda.empty_cache()
'''
import gc
del effic_model
gc.collect()
'''

'\nimport gc\ndel effic_model\ngc.collect()\n'

In [109]:
model.to("cpu")
createConfusionMatrix(test_loader, model)